# Competing phases 

To calculate the limiting chemical potentials we need to consider the energies of all competing phases. Doped does this by calling the `CompetingPhases` Class, which then queries Materials Project to obtain all the relevant competing phases. Please cross-check the competing phases with ICSD.

For the functionality to work correctly, you must have POTCARs set up for [pymatgen](https://pymatgen.org/installation.html#potcar-setup) and you will also need your Materials Project [API key](https://materialsproject.org/dashboard) set up in pymatgen. 

Doped assumes the so-called "molecule in a box" structures for the gaseous elemental phases H2, O2, N2, F2, Cl2, Br2 and I2. The molecule is placed in a  30Å x 30 Å x 30Å box, and relaxed with a 2x2x2 k-mesh

In [1]:
from doped.competing_phases import CompetingPhases

For example, if we were interested in the competing phases of La5Mn5O16, we would search across the La-Mn-O system like so: 

In [2]:
system = ['La', 'Mn', 'O']
cp = CompetingPhases(system, e_above_hull=0.03)

`cp.competing_phases` contains all the competing phases, their structures, magnetic moment and (MP-calculated GGA) band gaps. We can check how many there are by: 

In [4]:
print(len(cp.competing_phases))

30


From there you can either do your own thing with python / atomate / aiida to set up the calculations or you can use the doped to do that for you. 

The k-points convergence is done at GGA (PBEsol by default) and it set up to account for the magnetic moment convergence as well. All of this interfaces with [vaspup2.0](https://github.com/kavanase/vaspup2.0) so it's easy to use on the HPCs. You may want to change the default `ENCUT` or the general k-point densities that the convergence spans (5-60 kppvol for semiconductors & insulators and 40-120 for metals in steps of 5). It also uses `ISMEAR=-5` for metals by default and ignores convergence for molecules (because you really should just be using 2x2x2 kpoints) 

The kpoints convergence is set up with:

In [5]:
cp.convergence_setup(user_incar_settings={'ENCUT':550})

O2 is a molecule in a box, does not need convergence testing


/Users/katarina/miniconda3/envs/doped/lib/python3.8/site-packages/pymatgen/io/vasp/sets.py:424: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(


This creates a folder called `competing_phases` with all the stable&unstable competing phases. 

You would presumably also be interested in the folders for the relaxation of the said competing phases. By default doped will make this assuming a HSE06 INCAR and kppvol densities of 95 for metals and 45 for semiconductors. Obviously change the kpoints once you know the converged values but if you can't be arsed, those are good starting points. Doped can sort out different `SIGMA` and `ISMEAR`s needed for semiconductors and metals, and it can also set `NUPDOWN` in molecules (but not sure how well so double check that...). 

The relaxations can be set up with: 

In [6]:
cp.vasp_std_setup(user_incar_settings={'ENCUT':550})

/Users/katarina/miniconda3/envs/doped/lib/python3.8/site-packages/pymatgen/io/vasp/sets.py:424: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(


## Additional competing phases

So you've done your intrinsic defects and now you want to consider extrinsic doping. The addition of the new extrinsic species can also be handled with doped by the aptly named `AdditionalCompetingPhases` class. 

In [7]:
from doped.competing_phases import AdditionalCompetingPhases

In [8]:
system = ['La', 'Mn', 'O']
extrinsic_species = 'F'
acp = AdditionalCompetingPhases(system, extrinsic_species, e_above_hull=0.03) 

100%|██████████| 606/606 [00:02<00:00, 203.28it/s]


Doped can very cleverly tell what phases you've already calculated before and which ones should be added anew so it limits the number as we can see here:

In [10]:
len(acp.competing_phases)

23

The set up for convergence testing and relaxations is done in the exact same way as before: 

In [13]:
acp.convergence_setup(user_incar_settings={'ENCUT':550})

F2 is a molecule in a box, does not need convergence testing


/Users/katarina/miniconda3/envs/doped/lib/python3.8/site-packages/pymatgen/io/vasp/sets.py:424: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(


acp.vasp_std_setup(user_incar_settings={'ENCUT':550})

# Competing phases analyzer 


In [14]:
from doped.competing_phases import CompetingPhasesAnalyzer



## Read in data from vasprun.xml 

Once you've calculated all your very many competing phases you will want to analyse them. First you will want all your vaspruns neatly organised in some tree structure. To get them all off the HPCs recursively without including any other large files you can recursively rsync: 

```bash 
rsync -azvuR hpc:'path/to/the/base/folder/competing_phases/./formula_EaH_*/vasp_std/vasprun.xml' . 
```

where the `/./` indicates where you'd like to start the recurse from so you only keep the folder structure from the `formula_EaH_*` onwards. If you've done SOC calculations obviously change vasp_std to vasp_ncl or whatever you've called the folders. 

All analysis is done with aptly named `CompetingPhasesAnalyzer` and if you've used doped all you need to supply it is the 'pretty' formula of the system you're solving the chemical limits for (in this case that would be `'La5Mn5O16'` and the path to the base folder in which you have all your `formula_EaH_*/vasp_std/vasprun.xml` in. If you've used `vasp_ncl` (or anything else) instead of `vasp_std` you can set that as well. 

If you've not generated your competing phases inputs with doped, worry not because we've accounted for that too. You can generate a list of paths (or strings) to the vaspruns from using `pathlib` or `os`. 


In [16]:
system = 'La5Mn5O16'
cpa = CompetingPhasesAnalyzer(system)

In [ ]:
cpa.from_vaspruns(path='./lamno_competing_phases', 
                  folder='vasp_ncl', 
                  csv_fname='lamno_competing_phase_energies.csv')

The read in from vaspruns only needs to be done once, as the energies are saved to a csv file. 

An example of how to get all the vaspruns in one list if you've not used doped to generate them: 

In [ ]:
from pathlib import Path 
path = 'path/to/base'
all_paths = []
for p in path.iterdir():
    if not p.name.startswith('.'): 
        pp = p / 'relax' / 'vasprun.xml' 
        if pp.is_file():
            all_paths.append(pp)

## Read in data from a csv

As a sidenote you can also read in the data from a csv, as long as it contains the following headers: `'formula', 'energy_per_fu', 'energy', 'formation_energy'` 


In [ ]:
cpa.from_csv('path/to/csv.csv')

## Calculate the chemical potential limits

As easy as: 

In [ ]:
cpa.calculate_chempots()

This should save your chempots to a csv and also print them out for your viewing pleasure.

Eventually I'll get this working with extrinsic dopants but for now this is as good as its going to get

### CPLAP input


If you don't trust doped and pymatgen (ye of little faith), you can also create the `input.dat` file to use with [CPLAP](https://github.com/jbuckeridge/cplap). You can set the dependent variable, or leave it to doped to decide which one it will pick (all should yield the same numbers in higher order systems)

In [ ]:
cpa.cplap_input(dependent_variable='O') 

### Visualising the chemical potential limits 

For higher order systems, this interfaces really well with pymatgen's three-dimensional plotters, the list of total energies for the `ChemicalPotentialDiagram` are accessible from `cpa.pd_energies` and you can get the elemental energies to subtract from the `cpd.domains` by looping over `cpa.pd_energies` to find the ones with elemental formulas